<a href="https://colab.research.google.com/github/SonaDrocarova/ZLMA/blob/main/K%C3%B3pia_z%C3%A1pisn%C3%ADka_01ZLMA_zapoctova_uloha_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zadání zápočtové úlohy z předmětu 01ZLMA/01ZLIM v 2021/2022

Zápočtovou úlohu vypracujte na datech pacientů, u kterých byla indikována srdeční choroba. 

Originalni data jsou k nalezení: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

Různé analýzy a grafická vizualizace tohoto datasetu lze nalézt také na: https://www.kaggle.com/ronitf/heart-disease-uci (pozor jiné  popisy, nahrazeni NaN, ...)

Pro zápočtovou úlohu byla však data lehce modifikována a rozdělená z odkazu níže.

## 00 - Popis dat


    age: 
    sex:
        0: Female
        1: Male
    chest_pain_type: Chest Pain Type
        0: asymptomatic
        1: atypical angina
        2: non-anginal pain
        3: typical angina
    blood_pressure: Resting Blood Pressure: Person's resting blood pressure
    cholesterol: Serum Cholesterol in mg/dl
    blood_sugar: Fasting Blood Sugar
        0:Less Than 120mg/ml
        1: Greater Than 120mg/ml
    rest_ecg: Resting Electrocardiographic Measurement
        0: showing probable or definite left ventricular hypertrophy by Estes' criteria
        1: normal
        2: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    heart_rate: Max Heart Rate Achieved: Maximum Heart Rate Achieved
    ex_angina: Exercise Induced Angina
        1: Yes
        0: No
    st_depression: ST depression induced by exercise relative to rest
    st_slope: Slope of the peak exercise ST segment
        0: downsloping
        1: flat
        2: upsloping
    thal:  blood disorder called 'Thalassemia':
        1: fixed defect
        2: normal
        3: reversable Defect
    num_vessels: Number of Major Vessels: Number of major vessels colored by fluoroscopy


### Načtení a úprava dat



In [ ]:
library(tidyverse)
library(knitr)


In [ ]:
install.packages("GGally")
library(GGally)
install.packages("pander")
library(pander)
install.packages("glmnet")
library('glmnet')
#install.packages("ggthemes")
#library(ggthemes)

install.packages("lmtest")
library(lmtest)
install.packages("ggplot2")
library(ggplot2)
install.packages("ggpubr")
library(ggpubr)
#install.packages("car")
#library("car")
#install.packages("olsrr")
#library("olsrr")
#install.packages("ggfortify")
#library("ggfortify")
install.packages("data.table")
library(data.table)
install.packages("dplyr")
library(dplyr)
install.packages("tidyr")
library(tidyr)
install.packages("MASS")
library(MASS)



In [ ]:
install.packages('repr')
library(repr)


In [ ]:
data_train <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/heart_train.csv"
data_train  <- read.table(data_train, header = T, sep = ",")
head(data_train)

In [ ]:
data_test <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/heart_test.csv"
data_test  <- read.table(data_test, header = T, sep = ",")
head(data_test)

### Vytvoření agregované tabulky 

In [ ]:
data_table <- data_train %>%
   dplyr::select(age, sex, blood_pressure,disease) %>%
   mutate(age             = cut(age, breaks=c(-Inf, 44,60, Inf),labels=c("30-45","45-60","60-75")),
          blood_pressure  = cut(blood_pressure, breaks=c(-Inf, 120,130,140,Inf),labels=c("100-120","121-130","131-140","140-180"))) %>%
  group_by(age,blood_pressure) %>%
  summarise(n = n(),
         disease_yes = sum(disease),
         disease_no = n - sum(disease)
        )        
   data_table

## 01 - Grafické zobrazení dat (potřeba získat 2 / maximálně možno získat 3)

Použijte `data_train`. Nejen pro lepší práci a názornější grafy můžete kódové označení ve faktorových proměnných nahradit popisy ze zadání.

* Vyberte kategorické proměnné, převeďte je na faktory a přejmenujte z kódových názvů podle popisu dat.
* Zobrazte diskrétní proměnné pomocí histogramu, kde barevně rozlišíte mezi pacienty s vadou a bez vady srdce (target 0/1)
* Jednotlivé spojité proměnné zobrazte pomocí dvou boxplotu dle odezvy (s nemoci/bez nemoci) a pomoci scatterplotů mezi sebou, kde barevně oddělíte jednotlivé body opět podle odezvy (s nemocí srdce / bez nemoci srdce) 

**01-1** Vyberte kategorické proměnné, převeďte je na faktory a přejmenujte z kódových názvů podle popisu dat.

In [ ]:
data_train$sex <- factor(data_train$sex)
data_train$chest_pain_type<-factor(data_train$chest_pain_type,labels=c("asymptomatic","atypical","pain","typical"))
data_train$blood_sugar<-factor(data_train$blood_sugar,labels=c("less","more"))
data_train$rest_ecg<-factor(data_train$rest_ecg,labels=c("hypertrophy","normal","abnormality"))
data_train$ex_angina<-factor(data_train$ex_angina,labels=c("no","yes"))
data_train$st_slope<-factor(data_train$st_slope,labels=c("downsloping","flat","upsloping"))
data_train$thal<-factor(data_train$thal,labels=c('no',"fixed","normal","reversable"))
data_train$disease<-factor(data_train$disease,labels=c('no','yes'))

summary(data_train)

**01-2** Zobrazte diskrétní proměnné pomocí histogramu, kde barevně rozlišíte mezi pacienty s vadou a bez vady srdce (target 0/1):

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
ggplot(gather(data_train, key, value, -c(disease,sex,chest_pain_type,blood_sugar,rest_ecg,ex_angina,st_slope,thal,cholesterol,blood_pressure,heart_rate,st_depression)), aes(value, fill=disease)) + 
  geom_histogram(position = "identity", alpha = 0.4,bins = 20) + 
  facet_wrap(~key, scales = 'free')

**01-3** Jednotlivé spojité proměnné zobrazte pomocí dvou boxplotu dle odezvy (s nemoci/bez nemoci) a pomoci scatterplotů mezi sebou, kde barevně oddělíte jednotlivé body opět podle odezvy (s nemocí srdce / bez nemoci srdce):



In [ ]:
options(repr.plot.width=15, repr.plot.height=10)

ch<-ggplot(data_train, mapping=aes(disease, cholesterol,fill=disease)) +
  geom_boxplot(alpha=0.4) +
  xlab("Disease") + ylab("Cholesterol")
bp<-ggplot(data_train, mapping=aes(disease, blood_pressure,fill=disease)) +
  geom_boxplot(alpha=0.4) +
  xlab("Disease") + ylab("Blood pressure")
hr<- ggplot(data_train, mapping=aes(disease, heart_rate,fill=disease)) +
  geom_boxplot(alpha=0.4) +
  xlab("Disease") + ylab("Herat rate")
std<-ggplot(data_train, mapping=aes(disease, st_depression,fill=disease)) +
  geom_boxplot(alpha=0.4) +
  xlab("Disease") + ylab("Depression")
ggarrange(ch,bp,hr,std,
          ncol = 2, nrow = 2)

In [ ]:
options(repr.plot.width=20, repr.plot.height=15)

ch_bp<-ggplot(data_train, mapping=aes(cholesterol, blood_pressure, colour = disease)) +
  geom_point() +
  xlab("Cholesterol") + ylab("Blood pressure")+labs(colour="Disease")
ch_bp<-ggplot(data_train, mapping=aes(cholesterol, heart_rate, colour = disease)) +
  geom_point() +
  xlab("Cholesterol") + ylab("Heart rate")+labs(colour="Disease")
ch_std<-ggplot(data_train, mapping=aes(cholesterol, st_depression, colour = disease)) +
  geom_point() +
  xlab("Cholesterol") + ylab("Depression")+labs(colour="Disease")
bp_hr<-ggplot(data_train, mapping=aes(blood_pressure,heart_rate, colour = disease)) +
  geom_point() +
  xlab("Blood pressure") + ylab("Heart rate")+labs(colour="Disease")
bp_std<-ggplot(data_train, mapping=aes(blood_pressure, st_depression, colour = disease)) +
  geom_point() +
  xlab("Blood pressure") + ylab("Depression")+labs(colour="Disease")
hr_std<-ggplot(data_train, mapping=aes(heart_rate, st_depression, colour = disease)) +
  geom_point() +
  xlab("Heart rate") + ylab("Depression")+labs(colour="Disease")

ggarrange(ch_bp,ch_bp,ch_std,bp_hr,bp_std,hr_std,
          ncol = 3, nrow = 2)

## 02 - Logistická regrese agregovaná tabulková data (3/5)

použijte `data_table`





* Vytvořte odezvu pro logistický model binomického rozdělení a sestrojte nulový model. Jaká je ve zkoumané populaci  průměrná šance na srdeční chorobu a jaká je pravděpodobnost srdeční choroby? 

* Sestrojte model, kde závisí výskyt srdeční choroby pouze na tlaku. Je vliv tlaku statisticky významná proměnná na hladině významnosti 0.05? Pokud ano, zjistěte, kolikrát mají pacienti se srdečním tlakem mezi 140 a 180 vetší šanci na srdeční chorobu než pacienti s tlakem mezi 100 a 120.

* Sestrojte model, kde závisí výskyt srdeční choroby pouze na věku. Je věku  statisticky významná proměnná na hladině významnosti 0.01? Pokud ano, zjistěte, kolikrát mají pacienti ve věku mezi 60-75 a vetší šanci na srdeční chorobu než pacienti ve věku 45-60.

* Předpokládejte, že šance na srdeční chorobu roste exponenciálně s hodnotou tlaku a exponenciálně s věkem. Vytvořte příslušné numerické spojité prediktory, jakožto středy intervalů tlaku a věku. Sestrojte model, kde šance na srdeční chorobu bude záležet na numerických hodnotách tlaku a věku (bez interakce). Jaký je poměr šancí na srdeční chorobu mezi pacienty lišícími se o 10 let a majícími stejný tlak? 

* Otestujte předešlý model proti saturovanému modelu. Má tento test opodstatnění? Okomentujte jeho výsledek. 

**02-1** Vytvořte odezvu pro logistický model binomického rozdělení a sestrojte nulový model. Jaká je ve zkoumané populaci průměrná šance na srdeční chorobu a jaká je pravděpodobnost srdeční choroby?



In [ ]:
#data_table
dis<- cbind(data_table$disease_yes,data_table$disease_no)
model_min <- glm(dis~1,data=data_table,family=binomial)
summary(model_min)

*Šanca na srdcovú chorobu a pravdepodobnosť srdcovej choroby:*

In [ ]:
odds<-exp(coef(model_min))
c('Sanca:',odds) #sanca na srdcovu chorobu 
prob<-odds/(1+odds)
c('Pravdepodobnost:', prob) #pravdepodobnost srdcovej choroby

**02-2** Sestrojte model, kde závisí výskyt srdeční choroby pouze na tlaku. Je vliv 
tlaku statisticky významná proměnná na hladině významnosti 0.05?
Pokud ano, zjistěte, kolikrát mají pacienti se srdečním tlakem mezi 140 a 180 vetší šanci na srdeční chorobu než pacienti s tlakem mezi 100 a 120.

In [ ]:
model_pressure <- glm(dis~blood_pressure,data=data_table,family=binomial)
summary(model_pressure) #hodnoty tlaku medzi 140-180 su statisticky vzynamne

*Hodnoty tlaku medzi 140-180 su štatisticky vzýnamné na hladine 0,05. Kolikrát mají pacienti se srdečním tlakem mezi 140 a 180 vetší šanci na srdeční chorobu než pacienti s tlakem mezi 100 a 120:*

In [ ]:
data_pressure <- data_table %>% 
  mutate(blood_pressure = relevel(blood_pressure, ref = "100-120"))
model_100_120 <- glm(dis~blood_pressure,data=data_pressure,family=binomial)
#summary(model_100_120)
OR_pressure<-exp(coef(model_100_120)[4])
OR_pressure #sanca na srdcovu chorobu pre tlak 140-180 je 2,76*vyssia ako pri tlaku 100-120

*Šanca na srdcovú chorobu pre tlak 140-180 je 2,76-krát vyššia ako pri tlaku 100-120.*

**02-3** Sestrojte model, kde závisí výskyt srdeční choroby pouze na věku.
Je věku  statisticky významná proměnná na hladině významnosti 0.01? 
Pokud ano, zjistěte, kolikrát mají pacienti ve věku mezi 60-75 a vetší šanci na srdeční chorobu než pacienti ve věku 45-60.

In [ ]:
model_age <- glm(dis~age,data=data_table,family=binomial)
summary(model_age) 

*Věk je štatisticky významná proměnná na hladině významnosti 0.01. Kolikrát mají pacienti ve věku mezi 60-75 a vetší šanci na srdeční chorobu než pacienti ve věku 45-60:*

In [ ]:
data_age <- data_table %>% 
  mutate(age = relevel(age, ref = "60-75"))
model_60_75<- glm(dis~age,data=data_age,family=binomial)
OR_age<-exp(coef(model_60_75)[3])
OR_age 

*Šanca na srdcovú chorobu pre vek 45-60 je približne 0.7-krát nižšsia ako pri veku 60-75.*

**02-4** Předpokládejte, že šance na srdeční chorobu roste exponenciálně 
s hodnotou tlaku a exponenciálně s věkem. Vytvořte příslušné 
numerické spojité prediktory, jakožto středy intervalů tlaku 
a věku. Sestrojte model, kde šance na srdeční chorobu bude 
záležet na numerických hodnotách tlaku a věku (bez interakce).
Jaký je poměr šancí na srdeční chorobu mezi pacienty lišícími 
se o 10 let a majícími stejný tlak?

In [ ]:
data_table1<-data_table
levels(data_table1$age)<-c((30+45)/2,(45+60)/2,(60+75)/2)
data_table1$age<-as.numeric(levels(data_table1$age))[data_table1$age]
levels(data_table1$blood_pressure)<-c('110.0','126.5','136.5','160.0')
data_table1$blood_pressure<-as.numeric(levels(data_table1$blood_pressure))[data_table1$blood_pressure]
print(data_table1)
model_pressure_age <- glm(dis~blood_pressure+age,data=data_table1,family=binomial)
summary(model_pressure_age)

*Poměr šancí na srdeční chorobu mezi pacienty lišícími se o 10 let a majícími stejný tlak*:


In [ ]:
OR_pressure_age <- coef(model_pressure_age)
c <- 10
OR_pressure_age <- exp(c*OR_pressure_age[3])
OR_pressure_age#zvysi sa 1.4262*

**02-5** Otestujte předešlý model proti saturovanému modelu. Má tento test opodstatnění? Okomentujte jeho výsledek. 

In [ ]:
x<-as.factor(1:length(data_table1$blood_pressure))
SATmodel_pressure_age <- glm(dis~x,data=data_table1,family=binomial)
anova(model_pressure_age,SATmodel_pressure_age,test="Chisq")
#
#dev.sat <- deviance(SATmodel_pressure_age)
#dev.sat

*p-hodnota ANOVA testu je >0,05, zamietame hypotezu, ze saturovany model je štatisticky významne lepší ako model bez interakcie. Aby rozdelenie deviačnej štatistiky bolo blízke asymptotickému, je treba, aby $min(y_i)≥3$ a $min(n_i−y_i)≥3$.*

*Riadky tabulky nesplnaju $min(y_i)≥3$:*

In [ ]:
data_table1[rowSums(data_table1[,4] <3) > 0, ]

*Riadky tabulky nesplnaju $min(n_i−y_i)≥3$:*

In [ ]:
data_table1[rowSums(data_table1[,5] <3) > 0, ]

*Z predchádzajúcich tabuliek vidíme, že obidve podmienky sú porušené, test teda nemá opodstatnenie.*

## 03 - Poissonovská regrese na agregovaná tabulková data (4/7)

použijte  `data_table`


* Upravte tabulku do potřebného formátu a sestrojte čistě aditivní loglineární model pro četnosti pozorování ve skupinách, který předpokládá vzájemnou
nezávislost mezi všemi třemi skupinovými prediktory (věk, tlak, oneocnění).

* Na základě předchozího modelu, jaký je odhad šance na srdeční onemocnění mezi všemi vybranými pacienty a jaký je odhad pravděpodobnosti srdeční choroby?

* Sestavte model, který obsahuje všechny možné dvojné interakce mezi klasifikačními veličinami a porovnejte tento model s předchozím modelem bez interakcí. Je tento model statisticky významně lepší?

* Na základě předchozího modelu, jaký je odhad poměru šancí na srdeční onemocnění pacientů ve věku mezi 60-75 oproti pacientům ve věku 45-60?

* Sestavte saturovaný model a vypište odhady jeho parametrů parametry. Je tento model statisticky významně lepší než předchozí s dvojtými interakcemi? 

* Na základě saturovaného modelu, je ve všech věkových skupinách stejná souvislost mezi tlakem a onemocněním srdce?

* Ve které věkové kategorii je největší rozdíl v onemocnění srdce mezi
lidmi s tlakem menším než 120 a s lidmi s tlakem větším než 140?


**03-1** Upravte tabulku do potřebného formátu a sestrojte čistě aditivní loglineární model pro četnosti pozorování 
ve skupinách, který předpokládá vzájemnou
nezávislost mezi všemi třemi skupinovými 
prediktory (věk, tlak, oneocnění).

In [ ]:
install.packages("vcdExtra")
library(vcdExtra)

In [ ]:
tab<-expand.table(data_table,freq ='n' )

disease<-c()
for (i in 1:length(data_table$disease_yes)){
  disease<-append(disease,rep(1,data_table$disease_yes[i]))
  disease<-append(disease,rep(0,data_table$disease_no[i]))
}
tab$disease<-disease
tab<-tab[,-(3:4)]

table_form<-table(tab$blood_pressure, tab$age,tab$disease)
table_form<-ftable(table_form ,row.vars=3:2,col.vars = 1)
freq_tbl<-as.data.frame(table_form)
colnames(freq_tbl)<-c('disease','age','blood_pressure','Freq')
freq_tbl


*Aditívny loglineárny model.*

In [ ]:
mdl <- glm(Freq ~ blood_pressure + age+disease,data=freq_tbl, family=poisson(link="log"))
  summary(mdl)
#mdl_result<- c(deviance(mdl),df.residual(mdl))
#1-pchisq(q = mdl_result[1], df = mdl_result[2], lower.tail = F)
#vyjde p-hodnota...zamietame hypotezu ze su vzajomne nezavisle 

**03-2** Na základě předchozího modelu, jaký je odhad šance na srdeční 
onemocnění mezi všemi vybranými pacienty a jaký je odhad pravděpodobnosti 
srdeční choroby?

In [ ]:
odds_disease=exp(coef(mdl)[7])
odds_disease
probability <- (odds_disease)/(1+odds_disease)
probability

*Šanca ze ma chorobu je 0.82-krát menšia ako, že je zdravý. Pravdepodobnosť, že má chorobu je 0.45.*

**03-3** Sestavte model, který obsahuje všechny možné dvojné interakce
mezi klasifikačními veličinami a porovnejte tento model 
s předchozím modelem bez interakcí. 
Je tento model statisticky významně lepší?

In [ ]:
mdl_interactions <- glm(Freq ~.^2,data=freq_tbl, family=poisson(link="log"))
summary(mdl_interactions)

In [ ]:
anova(mdl,mdl_interactions,test="Chisq") 
c('AIC dvojne interakcie:',AIC(mdl_interactions))
c('BIC dvojne interakcie:',BIC(mdl_interactions))
c('AIC predchadzajuci model:',AIC(mdl))
c('BIC predchadzajuci model:',BIC(mdl))


*Model s dvojnými interakciami je statisticky vyznamne lepsi na základe prevedeného ANOVA testu. Hodnoty AIC a BIC sú pre tento model rovnako lepšie. Model s dvojnými interakciami je teda lepši oproti predošlému modelu.*


**03-4** Na základě předchozího modelu, jaký je odhad poměru šancí na srdeční onemocnění pacientů ve věku mezi 60-75 oproti pacientům ve věku 45-60?

In [ ]:
freq_age <- freq_tbl %>% 
  mutate(age = relevel(age, ref = "45-60"))
mdl_interactions_age <- glm(Freq ~.^2,data=freq_age, family=poisson(link="log"))
summary(mdl_interactions_age)

*Odhad poměru šancí na srdeční onemocnění pacientů ve věku mezi 60-75 oproti pacientům ve věku 45-60:*

In [ ]:
OR<-exp(coef(mdl_interactions_age)[4])
OR

**03-5** Sestavte saturovaný model a vypište odhady jeho 
parametrů parametry. Je tento model statisticky 
významně lepší než předchozí s dvojtými interakcemi?

In [ ]:
mdl_sat <- glm(Freq ~age*blood_pressure*disease,data=freq_tbl, family=poisson(link="log"))
tibble(coef=names(coef(mdl_sat)),
           coef_value=coef(mdl_sat),
           exp_coef_value=exp(coef(mdl_sat)))
summary(mdl_sat)

In [ ]:
anova(mdl_interactions,mdl_sat,test="Chisq")

*Model nie je štatisticky významne lepší než model s dvojnými interakciami*

**03-6** Na základě saturovaného modelu, je ve všech věkových 
skupinách stejná souvislost mezi tlakem a onemocněním srdce?

*Súvislosť medzi tlakom a ochorením srdca budeme skúmať vymazaním všetkých interakcií tlaku a ochorenia v saturovanom modeli a následným testovaním, hypotézy, že model dobre popisuje dáta.*

In [ ]:
model_pressure_disease<-glm(Freq~age*blood_pressure*disease-blood_pressure:age:disease-blood_pressure:disease,data=freq_tbl, family=poisson(link="log"))

In [ ]:
mdl_result<- c(deviance(model_pressure_disease),df.residual(model_pressure_disease))
pchisq(q = mdl_result[1], df = mdl_result[2], lower.tail = F)

*p-hodnota testu je 0.298472932401528. Hypotézu, že model dobre popisuje data sme na hladine významnosti 0.05 nezamietli, preto zamietam hypotezu suvislosti tlaku a choroby.*

**03-7** Ve které věkové kategorii je největší rozdíl v 
onemocnění srdce mezi lidmi s tlakem menším než 
120 a s lidmi s tlakem větším než 140?

*Z tabuľky som odčítala počty ľudí ktorí majú v danej vekovej kategórii chorobu a tlak nad 120 od počtu ľudí v rovnakej vekovej kategórii s choroboou a tlakom nad 140. Tieto hodnoty som ešte pre každý vek normalizovala celkovým počtom ľudí ktorí chodobu majú v danej vekovej kategórii.*

In [ ]:
disease_under<-table_form[,1][4:6]
disease_over<-table_form[,4][4:6]

c('30-45',abs(disease_under-disease_over)[1]/sum(table_form[4,]))
#'pod 120:',disease_under[1]/sum(table_form[4,]),'nad 140:',disease_over[1]/sum(table_form[4,]), 'rozdiel:',abs(disease_under-disease_over)[1]/sum(table_form[4,]))
c('45-60',abs(disease_under-disease_over)[2]/sum(table_form[5,]))
#disease_under[2]/sum(table_form[5,]),'nad 140:',disease_over[2]/sum(table_form[5,]), 'rozdiel:',abs(disease_under-disease_over)[2]/sum(table_form[5,]))
c('60-75',abs(disease_under-disease_over)[3]/sum(table_form[6,]))
#'pod 120:',disease_under[3]/sum(table_form[6,]),'nad 140:',disease_over[3]/sum(table_form[6,]), 'rozdiel:',



*Vo vekovej kategórii 30-45 rokov je najväčší rozdiel medzi ochorením srdca pri tlaku menšom ako 120 a väčšiom ako 140.*

## 04 - Logistická regrese - statistika (5/8)

Nyní již používejte `data_train`

* Vypište kontingenční tabulku pro proměnné pohlaví (`sex`) a onemocnění (`disease`). Ručně z tabulky spočtěte empirický poměr šancí na onemocnění srdce (muž vs. žena) a pravděpodobnost onemocnění pro ženu a muže. Výsledek porovnejte s výsledky z logistické regrese, kde jediným prediktorem bude právě proměnná `sex` a odezvou proměnná `disease`. Pro poměr šancí uveďte i 95% konfidenční interval a okometujte, zdali mají ženy signifikantně menší šanci na onemocnění srdce.

* Vypište kontingenční tabulku pro proměnné typ bolesti (`chest_pain_type`) a onemocnění (`disease`). Ručně z tabulky spočtěte empirický poměr šancí na onemocnění srdce (type 0: asymptomatic, proti ostatním) a pravděpodobnosti onemocnění pro jednotlivé typy. Výsledek porovnejte s výsledky z logistické regrese, kde jediným prediktorem bude právě proměnná `chest_pain_type` a odezvou proměnná `disease`. Pro poměr šancí uveďte i 95% konfidenční interval a okometujte, zdali mají pacienti s asymptomatickou bolestí signifikantně menší šanci na onemocnění srdce oproti ostatním typům bolestí.

* Sestavte model, kde použijete všechny dostupné proměnné (jak faktorové tak numerické). Pomocí deviančních testů model postupně redukujte. Výsledný model porovnejte s modelem, který byste obdrželi při použití automatického výběru s funkcí `step()`.

* Pro vybraný model spočtěte šanci na srdeční onemocnění pro muže oproti ženám, včetně 95% konfidenčních intervalů. Stejně tak pro asymptomaticky typ bolesti na hrudi proti ostatním. Jak se výsledek změnil oproti jednoduchým modelům a jak byste tuto změnu vysvětlili?

* (2b) Použijte váš model a spočtěte odhad pravděpodobnosti srdeční choroby pro testovací data vykreslete pro proměnou `blood_pressure` predikční konfidenční intervaly. 
Nápověda krom cvičení, také třeba zde: https://fromthebottomoftheheap.net/2017/05/01/glm-prediction-intervals-i/

* (2b) Na základě trénovacích dat zvolte vhodný treshold pro určení má nemoc / nemá nemoc a na testovacích datech spočtěte Accuracy a vykreslete ROC křivku.





**04-1** Vypište kontingenční tabulku pro proměnné pohlaví (`sex`)
a onemocnění (`disease`). Ručně z tabulky spočtěte empirický 
poměr šancí na onemocnění srdce (muž vs. žena) a pravděpodobnost 
onemocnění pro ženu a muže. Výsledek porovnejte s výsledky z 
logistické regrese, kde jediným prediktorem bude právě proměnná 
`sex` a odezvou proměnná `disease`. Pro poměr šancí uveďte 
i 95% konfidenční interval a okometujte, zdali mají ženy 
signifikantně menší šanci na onemocnění srdce.

In [ ]:
sex_disease <-table(data_train$sex, data_train$disease)
sex_disease

*Empirický pomer šancí na ochorenie muz vs zena:*


In [ ]:
sex_disease[1]*sex_disease[4]/(sex_disease[2]*sex_disease[3])

*Muži majú 3.5-krát väčšiu šancu na heart disease.*

In [ ]:
#pravdepod onemoc muz,zena

zena<-sex_disease[3]/(sex_disease[1,1]+sex_disease[1,2])
muz<-sex_disease[4]/(sex_disease[2,1]+sex_disease[2,2])
c('pravdepodobnost ze ma zena srdcovu chorobu:',zena)
c('pravdepodobnost ze ma muz srdcovu chorobu:',muz)


In [ ]:
#model
c('pravdepodobnosti z modelu:')
model_sex_disease<-glm(disease~sex,data=data_train,family = binomial(link = "logit"))
glm_probs = data.frame(probs = predict(model_sex_disease, type="response"))
head(unique(glm_probs))
c('pomer sanci z modelu:')
exp(cbind(coef(model_sex_disease), confint(model_sex_disease)))

*Pomer šancí vypočítaný z modelu je približne rovnaký ako empirický pomer šancí. Rovnako pre pravdepodobnosti. Z 95% konfidenčného intervalu vidíme (neobsahuje 1), že šanca na ochorenie pre ženy je signifikantne menšia ako pre mužov.*

**04-2** Vypište kontingenční tabulku pro proměnné typ bolesti 
(`chest_pain_type`) a onemocnění (`disease`). Ručně z 
tabulky spočtěte empirický poměr šancí na onemocnění 
srdce (type 0: asymptomatic, proti ostatním) a 
pravděpodobnosti onemocnění pro jednotlivé typy.
Výsledek porovnejte s výsledky z logistické regrese, 
kde jediným prediktorem bude právě proměnná `chest_pain_type`
a odezvou proměnná `disease`. Pro poměr šancí uveďte i 95% 
konfidenční interval a okometujte, zdali mají pacienti s 
asymptomatickou bolestí signifikantně menší šanci na 
onemocnění srdce oproti ostatním typům bolestí.

In [ ]:
chest_disease <-table(data_train$chest_pain_type, data_train$disease)
chest_disease

In [ ]:
#pravdepod onemoc 

asympt<-chest_disease[5]/sum(chest_disease[1,])
atyp<-chest_disease[6]/sum(chest_disease[2,])
pain<-chest_disease[7]/sum(chest_disease[3,])
typ<-chest_disease[8]/sum(chest_disease[4,])
c('pravdepodobnosht ochorenia pri type asymptomatic',asympt)
c('pravdepodobnosht ochorenia pri type atupical',atyp)
c('pravdepodobnosht ochorenia pri type pain',pain)
c('pravdepodobnosht ochorenia pri type typical',typ)

In [ ]:
#empiricky pomer sanci na onemoc pri chest pain 0
odds_asympt<-asympt/(1-asympt)
odds_atyp<-atyp/(1-atyp)
odds_pain<-pain/(1-pain)
odds_typ<-typ/(1-typ)
chest_pain_atypical<-odds_atyp/odds_asympt
chest_pain<-odds_pain/odds_asympt
chest_pain_typical<-odds_typ/odds_asympt
c('pomer sanci pre typ asymptomatic proti ostatnym typom bolesti:')
tibble(chest_pain_atypical,chest_pain,chest_pain_typical)

In [ ]:
#model
model_chest_pain<-glm(disease~chest_pain_type,data=data_train,family = binomial(link = "logit"))
c('pomer sanci z modelu:')
exp(cbind(coef(model_chest_pain), confint(model_chest_pain)))
#exp(sum(coef(model_chest_pain)[2:3]))

*Pacienti s typom bolesti asymptomatic majú signifikantne vyššiu šancu na ochorenie srdca ako pacienti s ostatnými typmi bolesti. Pomery šancí z modelu vychádzajú rovnaké ako z výpočtu pomocou empirických dát.*

In [ ]:
glm_probs = data.frame(probs = predict(model_chest_pain, type="response"))
head(unique(glm_probs))

*Pravdepodobnosti z modelu majú rovnakú hodnotu ako pravdepodobnosti pomocou tabuľky.*

**04-3** Sestavte model, kde použijete všechny dostupné proměnné
(jak faktorové tak numerické). Pomocí deviančních testů 
model postupně redukujte. Výsledný model porovnejte s modelem, 
který byste obdrželi při použití automatického výběru s funkcí 
`step()`.

In [ ]:
mdl_1 <- glm(disease~.-disease,family = binomial('logit'),data=data_train)
summary(mdl_1)
drop1(mdl_1, test="Chisq")
mdl_2 <- update(mdl_1, ~. - age - blood_sugar- cholesterol- rest_ecg- ex_angina)  
summary(mdl_2)
drop1(mdl_2, test="Chisq")
c(mdl_1$aic,mdl_2$aic)
anova(mdl_2,mdl_1, test="Chisq")#uplny nie je vyzn. lepsi tento 
#model by som teda zobrala

*Model pomocou step.*

In [ ]:
mdl_step <- step(mdl_1,direction="backward")
summary(mdl_step)

*Model, ktorý som dostal vypúšťaním premenných pomocou deviačných testov je rovnaký ako model funkciou step. Ďalej používam model step.*

**04-4** Pro vybraný model spočtěte šanci na srdeční onemocnění
pro muže oproti ženám, včetně 95% konfidenčních intervalů. 
Stejně tak pro asymptomaticky typ bolesti na hrudi proti
ostatním. Jak se výsledek změnil oproti jednoduchým modelům 
a jak byste tuto změnu vysvětlili?

In [ ]:
c('pomocou vybraneho modelu:')
exp(cbind(coef(mdl_step), confint(mdl_step)))[(2:5),]
c('pomocou jednoducheho modelu')
exp(cbind(coef(model_sex_disease), confint(model_sex_disease)))
exp(cbind(coef(model_chest_pain), confint(model_chest_pain)))[(2:4),]

*Šanca na chorobu u mužov je v modeli z predchádzajúcej úlohy ešte vyššia. Muži majú v oboch modeloch signifikantne väčšiu šancu na ochorenie srdca ako ženy. Šanca na srdečnú chorobu pri asymptomatickej bolesti je v novom modely približne 3.3-krát väčšia kdežto v jednoduchom modely bola až 13.05-krát väčšia než pri atypickej bolesti. Ostatné Pomery šancí sa menili menej výrazne. Stále sú však vo všetkých prípadoch šance na ochorenie signifikantne väčšie pri asymptomatickej bolesti. Hodnoty sa zmnenili z toho dôvodu, že sa zmenili parametre modelu, z ktorých je dalej počítaný pomer šancí.*

**04-5** (2b) Použijte váš model a spočtěte odhad pravděpodobnosti 
srdeční choroby pro testovací data vykreslete pro proměnou 
blood_pressure predikční konfidenční intervaly. 

In [ ]:

data_test$sex <- factor(data_test$sex,labels=c("female","male"))
data_test$chest_pain_type<-factor(data_test$chest_pain_type,labels=c("asymptomatic","atypical","pain","typical"))
data_test$blood_sugar<-factor(data_test$blood_sugar,labels=c("less","more"))
data_test$rest_ecg<-factor(data_test$rest_ecg,labels=c("hypertrophy","normal"))
data_test$ex_angina<-factor(data_test$ex_angina,labels=c("no","yes"))
data_test$st_slope<-factor(data_test$st_slope,labels=c("downsloping","flat","upsloping"))
data_test$thal<-factor(data_test$thal,labels=c('no',"fixed","normal"))
data_test$disease<-factor(data_test$disease,labels=c('no','yes'))

data_test_pred<-data_test%>%
  mutate(fitted = predict(mdl_step, newdata = data_test, type = "response"),
         predict = as.factor(ifelse(fitted > 0.5, "yes", "no")))
head(data_test_pred)

ilink <- family(mdl_step)$linkinv
quant_t <- qt(0.025, df = df.residual(mdl_step), lower.tail = FALSE)
quant_n <- qnorm(0.005, lower.tail = FALSE)

pd<-with(data_test,data.frame(blood_pressure = seq(min(blood_pressure), max(blood_pressure),
                                               length = 100)))
#ostane premenne nasatvime ako konstantne
pd$sex<-rep('male',100)
pd$chest_pain_type<-rep('asymptomatic',100)
pd$heart_rate<-rep(140,100)
pd$st_depression<-rep(0,100)
pd$st_slope<-rep('flat',100)
pd$num_vessels<-rep(0,100)
pd$thal<-rep('normal',100)


pd<-cbind(pd, predict(mdl_step, pd, type = "link", se.fit = TRUE))

plt <- ggplot(pd, aes(x = blood_pressure, y = ilink(fit))) +
  geom_point() 
pd <- pd %>%
  mutate(fit_link = predict(mdl_step, pd, se.fit = TRUE)$fit,
         se_link  = predict(mdl_step, pd, se.fit = TRUE)$se.fit,
         fit  = ilink(fit_link),
         fit_ul = ilink(fit_link + (quant_n * se_link)),
         fit_ll = ilink(fit_link - (quant_n * se_link)))
## show

plt + geom_ribbon(data = pd,
                  aes(ymin = fit_ll, ymax = fit_ul),fill = "steelblue2",
                  alpha = 0.1)

**04-6** (2b) Na základě trénovacích dat zvolte vhodný treshold 
pro určení má nemoc / nemá nemoc a na testovacích datech 
spočtěte Accuracy a vykreslete ROC křivku.

In [ ]:
install.packages("pROC")
library(pROC)       

In [ ]:
#do datasetu pridame fitted a prehodime hodnoty disease naspat do tvaru 0,1
data_train_pred1<-data_train%>%
  mutate(fitted = predict(mdl_step, newdata = data_train, type = "response"),
        disease = as.numeric(ifelse(disease=="yes", 1, 0)))
head(data_train_pred1)

In [ ]:
install.packages('rfUtilities')
library('rfUtilities')
install.packages("caret")
library(caret)
install.packages("InformationValue")
library(InformationValue)
install.packages("ISLR")
library(ISLR)

In [ ]:
cutoff<-optimalCutoff(data_train_pred1$disease, data_train_pred1$fitted)
c('threshold:',cutoff)
data_test_pred<-data_test%>%
  mutate(fitted = predict(mdl_step, newdata = data_test, type = "response"),
          predict = as.factor(ifelse(fitted > cutoff, "yes", "no")))
head(data_test_pred)


In [ ]:
plot.roc(data_test_pred$disease, data_test_pred$fitted,     
         percent = TRUE,
         print.auc=TRUE,                    
         main = "Partial AUC")

In [ ]:
#accuracy(data_train_pred$predict, data_train_pred$disease)
accuracy(data_test_pred$predict, data_test_pred$disease)$confusion
c('Accuracy:',accuracy(data_test_pred$predict, data_test_pred$disease)$PCC)


## 05 - Logistická regrese -  strojové učení (1/3)

* Sestavte workflow (pipeline) na trenovacích datech pro logistickou regresi s použitím regularizace (elestic-net), která bude obsahovat 
 *   Přípravu proměnných, transformaci, one-hot encoding, normalizaci, ... (Možno použít recipe z tidy models)
 *   Hledání "optimálního" nastavení hyperparametru v regularizaci. 
 *   k-fold cross validaci

* Pomocí předchozí pipline/workflow vyberte hodnotu hyperparametru. Pokud chceme uvedený algoritmus použít k detekci pacientů s nemocí srdce, jakou statistiku bychom měli sledovat abychom omylem nepropustili nemocného domů jako zdravého? 

* Spočtěte a porovnejte statistiky používané při binární klasifikaci na trénovacích a testovacích datech. Vykreslete ROC křivku a spočtěte plochu pod ní jak pro testovací tak trénovací data. Co můžeme říci o kvalitě modelu z této sekce 05 a modelu z předchozí 04?




**05-1** Sestavte workflow (pipeline) na trenovacích datech pro logistickou regresi s použitím regularizace (elestic-net), která bude obsahovat 
 *   Přípravu proměnných, transformaci, one-hot encoding, normalizaci, ... (Možno použít recipe z tidy models)
 *   Hledání "optimálního" nastavení hyperparametru v regularizaci. 
 *   k-fold cross validaci

In [ ]:
install.packages("tidymodels")
library(tidymodels)  


*Priprava premennych:*

In [ ]:
split <- initial_split(data_train, strata = disease, prop = 0.8)
train_d <- training(split)
val_d <- validation_split(data_train, 
                            strata = disease, 
                            prop = 0.80)
mod_lr <- 
  logistic_reg(penalty = tune(), mixture = 1) %>% 
  set_engine("glmnet")
recipe0 <- recipe(disease ~ ., data = train_d) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>% 
    step_dummy(all_nominal(), -all_outcomes())
train <- recipe0 %>%
            prep() %>%
            juice()


In [ ]:
recipe1 <- recipe0 %>%
    update_role(age, sex, chest_pain_type,cholesterol, blood_sugar, rest_ecg,
                blood_pressure, heart_rate, ex_angina, st_depression, num_vessels, st_slope,thal,
                new_role = "predictor") %>%
    update_role(disease, new_role = "outcome")
workflow_lr <- 
  workflow() %>% 
  add_recipe(recipe1) %>% 
  add_model(mod_lr) 

In [ ]:
tune_grid<-tibble(penalty = 10^seq(-4, -1, length.out = 50))
results <- 
  workflow_lr %>% 
  tune_grid(val_d,
            grid = tune_grid,
            control = control_grid(save_pred = TRUE, verbose = TRUE),
            metrics = metric_set(roc_auc, accuracy,specificity,sensitivity))


In [ ]:
collect_metrics(results) %>%
  pivot_wider(names_from = .metric, values_from = mean ) %>%
  arrange(desc(roc_auc))
lr_best <-
  results %>% 
  collect_metrics() %>% 
  filter(.metric == "accuracy") %>% 
  filter(mean == max(mean)) %>% 
  slice(1)
c('optimalny hyperparameter:')
lr_best

*k-fold cross validace:*

In [ ]:
train_fold <- vfold_cv(train_d,strata = "disease", v = 5)
summary(train_fold)

lr_fold <- workflow_lr %>%
  tune_grid(
    resamples = train_fold,
    grid = tune_grid,
    metrics = metric_set(roc_auc)
  )

In [ ]:
collect_metrics(lr_fold) %>%
 pivot_wider(names_from = .metric, values_from = mean ) %>%
 arrange(desc(roc_auc))

**05-2** Pomocí předchozí pipline/workflow vyberte hodnotu hyperparametru. Pokud chceme uvedený algoritmus použít k detekci pacientů s nemocí srdce, jakou statistiku bychom měli sledovat abychom omylem nepropustili nemocného domů jako zdravého?

In [ ]:
best <- 
  lr_fold %>% 
  collect_metrics() %>% 
  arrange(desc(mean)) %>% 
  slice(1)
c('vybrana hodnota hyperparametru podla predchadzajucej ulohy:')
best$penalty

*Aby sme omylom neprepustili nemocného človeka, budeme sledovať špecificitu.*

**05-3** Spočtěte a porovnejte statistiky používané při binární klasifikaci na trénovacích a testovacích datech. Vykreslete ROC křivku a spočtěte plochu pod ní jak pro testovací tak trénovací data. Co můžeme říci o kvalitě modelu z této sekce 05 a modelu z předchozí 04?

In [ ]:
mdl_final <- 
  logistic_reg(penalty = best$penalty, mixture = 1) %>% 
  set_engine("glmnet")

workflow_final <- 
  workflow() %>% 
  add_recipe(recipe1) %>% 
  add_model(mdl_final) 

results_final<-last_fit(workflow_final,split,metrics=metric_set(roc_auc, accuracy,specificity,sensitivity))
results_final$.metrics


In [ ]:
lr_auc <- 
  results %>% 
  collect_predictions(parameters = best) %>% 
  roc_curve(disease, .pred_no) %>% 
  mutate(model = "Logistic Regression - glmnet")

autoplot(lr_auc)

*Z výsledkov vidíme, že dostávame pre model pomerne vysoké hodnoty pre testovacie metriky. Model z predošlej úlohy má podstatne nižšiu prestnosť.*

## Poznámky k odevzdání a vypracování

* Úkol vypracujte samostatně. Pokud se budete s někým radit, nezapomeňte u dané odpovědi uvést zdroj vašich informací. 

* Úkol můžete odevzdávat jako spustitelný jupyter notebook na google colab (stačí odevzdat link na github). Před odevzdáním ZKONTROLUJTE, že buňky na sebe navazují a že lze NB postupně projít a jednotlivé buňky nezahlásí chybu.

* Deadline pro odevzdání je stanoven na sředu 22.6.2022.

* Není možno odevzdání opakovat. Odevzdaný úkol zkontroluji a buď zápočet získáte, nebo ne.

* Pokud se rozhodnete na někkteré otázky neodpovídat, uveďte to a počítejte s tím, že v dané sekci ztrácíte body a odpovědi z dalších otázek nemusí stačit na minimum za danou sekci.

* V případě otázek se ptejte veřejně v general kanále v MS Teams. Ostatní mohou mít podobné problémy a tudíž pokud je někde například nejasné zadání, tak ať mají všichni stejnou příležitost.

* Možno psát a odevzdat v Pythonu.
